In [1]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from dataConfig import Config
from data import SimulationData
import torch
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader  

from torch_geometric.data import Data, Batch
from models import DeepONet
from dataset import MultiColDataset

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

seed = 42
set_seed(seed)

Random seed set as 42


In [2]:
config = Config()

In [3]:
data = SimulationData(config)
data.load_features('data/features.pkl')

In [4]:
data.features.keys()

dict_keys(['W16X100', 'W16X26', 'W16X31', 'W16X36', 'W16X40', 'W16X45', 'W16X50', 'W16X57', 'W16X67', 'W16X77', 'W16X89', 'W18X106', 'W18X119', 'W18X130', 'W18X35', 'W18X40', 'W18X46', 'W18X50', 'W18X55', 'W18X60', 'W18X65', 'W18X71', 'W18X76', 'W18X86', 'W18X97', 'W21X101', 'W21X111', 'W21X122', 'W21X132', 'W21X44', 'W21X48', 'W21X50', 'W21X55', 'W21X57', 'W21X62', 'W21X68', 'W21X73', 'W21X83', 'W21X93', 'W24X103', 'W24X104', 'W24X117', 'W24X131', 'W24X146', 'W24X55', 'W24X62', 'W24X68', 'W24X76', 'W24X84', 'W24X94', 'W27X102', 'W27X114', 'W27X129', 'W27X146', 'W27X161', 'W27X84', 'W27X94', 'W30X108', 'W30X116', 'W30X124', 'W30X132', 'W30X148', 'W30X173', 'W30X90', 'W30X99', 'W33X118', 'W33X130', 'W33X141', 'W33X152', 'W33X169', 'W33X201', 'W36X135', 'W36X150', 'W36X160', 'W36X170', 'W36X182', 'W36X194', 'W36X210', 'W40X149', 'W40X167', 'W40X183', 'W40X211'])

In [5]:
data.features["W16X100"].keys()

dict_keys(['reserve_capacities', 'loading_protocols', 'coordinate_features', 'deviation_features', 'curvature_features', 'edge_index', 'edge_features', 'd', 't_w', 'b_f', 't_f', 'edge_norms', 'In-distribution test', 'Out-of-distribution test'])

In [6]:
data.features["W16X100"]["In-distribution test"]

False

In [9]:
# --- Create multi-column dataset ---
dataset = MultiColDataset(data)

# Optionally split into train/val
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# --- Model, optimizer, loss ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepONet(branch_in=7, trunk_in=4, out_dim=2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = torch.nn.MSELoss()

# --- Training loop ---
n_epochs = 50
for epoch in range(n_epochs):
    model.train()
    total_loss = 0.0

    for graphs, trunk_inputs, targets in train_loader:
        graphs = graphs.to(device)
        trunk_inputs = trunk_inputs.to(device)
        targets = targets.to(device)

        preds = model(graphs, trunk_inputs)

        loss = criterion(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * graphs.num_graphs

    avg_train_loss = total_loss / len(train_dataset)

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for graphs, trunk_inputs, targets in val_loader:
            graphs = graphs.to(device)
            trunk_inputs = trunk_inputs.to(device)
            targets = targets.to(device)

            preds = model(graphs, trunk_inputs)
            val_loss += criterion(preds, targets).item() * graphs.num_graphs

    avg_val_loss = val_loss / len(val_dataset)

    print(f"Epoch {epoch+1:03d} | Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f}")

KeyboardInterrupt: 

In [ ]:
# 70 nodes, x shape :(70,dim_features)